#### Import modules

In [7]:
import sys
import os
from numpy import arange
from pandas import read_csv, DataFrame, Series
from matplotlib.pyplot import savefig
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

utils_path = os.path.abspath(os.path.join('..', 'utils'))
sys.path.append(utils_path)

from dslabs_functions import ts_aggregation_by, series_train_test_split, plot_forecasting_eval, plot_forecasting_series

#### Load data

In [8]:
data_path = os.path.abspath(os.path.join('..','..', 'data_copy/forecast_ny_arrests.csv'))
if not os.path.exists(data_path):
    print("File not found")
    data_path = os.path.abspath(os.path.join('..','..', 'data/f_s_forecast_ny_arrests.csv'))

def load_data(data_path):
    data: DataFrame = read_csv(
        data_path,
        index_col="Date",
        sep=";",
        decimal=".",
        parse_dates=True,
        infer_datetime_format=True,
    )
    return data

all_data = load_data(data_path)
file_tag = "ARREST"
target = "Manhattan"



data = all_data[[target]]


File not found


In [9]:
from pandas import DataFrame

def dataframe_train_test_split(data: DataFrame, trn_pct: float = 0.90) -> tuple[DataFrame, DataFrame]:
    trn_size = int(len(data) * trn_pct)
    train = data.iloc[:trn_size]
    test = data.iloc[trn_size:]
    return train, test


#### Preparation

In [10]:
import pandas as pd

def data_preparation(data):
    # Missing value 
    data: DataFrame = data.dropna(how="any", inplace=False) 
    # Train/test split
    train, test = dataframe_train_test_split(data)   
    
    # Scaling
    transf: MinMaxScaler = MinMaxScaler(feature_range=(0, 1), copy=True).fit(train)
    train_scaled = DataFrame(transf.transform(train), index=train.index, columns=train.columns)
    test_scaled = DataFrame(transf.transform(test), index=test.index, columns=test.columns) 

    # Smoothing
    WIN_SIZE = 10
    train_smooth: Series = train_scaled.rolling(window=WIN_SIZE).mean()
    train_smooth = train_smooth.dropna()

    # Differentiation
    train_diff: Series = train_smooth.diff()
    train_diff = train_diff.dropna()
    test_diff: Series = test_scaled.diff()
    test_diff = test_diff.dropna()

    return train_diff, test_diff
    
    
train, test = data_preparation(data)

train_all, test_all = data_preparation(all_data)


print(train.columns, test.columns)
print(train_all.columns, test_all.columns)

Index(['Manhattan'], dtype='object') Index(['Manhattan'], dtype='object')
Index(['Bronx', 'Brooklyn', 'Queens', 'StatenIsland', 'Manhattan'], dtype='object') Index(['Bronx', 'Brooklyn', 'Queens', 'StatenIsland', 'Manhattan'], dtype='object')
